# Part 3 - Stage 2 Data Manipulation

The purpose of this library is to manipulate stage 2 data into useable training data. 

The inputs you need for this are:
/data/MNCAATourneySeeds.csv
/data/MRegularSeasonDetailedResults.csv

## Library Imports

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pkg_resources

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

In [3]:
cwd = os.getcwd()

## Data Manipulation

### Regular Season Data Analysis

In [29]:
# Existing code from data manipulation section. Only run if needed. 
cwd = os.getcwd()

# tourney_cresults = pd.read_csv(cwd + '/data/MNCAATourneyCompactResults.csv')
# seeds = pd.read_csv(cwd + '/data/MNCAATourneySeeds.csv')
seeds = pd.read_csv(cwd + '/data_stage2/MNCAATourneySeeds.csv')
season_dresults = pd.read_csv(cwd +'/data_stage2/MRegularSeasonDetailedResults.csv')

seeds['Seed'] = pd.to_numeric(seeds['Seed'].str[1:3],downcast='integer',errors='coerce')
currentYear = 2023
targetYear = 2003
# tourney_cresults = tourney_cresults.loc[tourney_cresults['Season'] >= targetYear]

training_set = pd.read_csv("training_set.csv")
record = pd.read_csv('record.csv')

# seeds['Seed'] =  pd.to_numeric(seeds['Seed'].str[1:3], downcast='integer',errors='coerce')


def delta_seed(row):
    cond = (seeds['Season'] == row['Season'])
    return seeds[cond & (seeds['TeamID'] == row['Team1'])]['Seed'].iloc[0] - seeds[cond & (seeds['TeamID'] == row['Team2'])]['Seed'].iloc[0]

# function to, given a row, calculate what the difference between the two seeds was. 
#Function to look up 
def delta_winPct(row):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1]['wins']/record[cond1]['games']).mean() - (record[cond2]['wins']/record[cond2]['games']).mean()

def get_points_against(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'LScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'WScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_points_for(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'WScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'LScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_remaining_stats(row, field):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'W' + field
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'L'+ field
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum()
    return retVal

def delta_stat(row, field):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1][field]/record[cond1]['games']).mean() - (record[cond2][field]/record[cond2]['games']).mean()

In [24]:
# def delta_seed(row):
#     team1Seed = seeds[seeds['TeamID'] == row['Team1']]['Seed']
#     seeds
#     return team1Seed

In [23]:
# seeds[seeds['TeamID']==1207]['Seed']

0        1
83       4
144      1
199      8
256      1
338      3
439      8
469      6
616      9
693      6
705      2
825     10
1049    10
1387     7
1415     2
1497     2
1644     3
1730     6
1760     3
1828     2
1966     4
2298    12
Name: Seed, dtype: int8

In [30]:
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2485,2023,12,1433
2486,2023,13,1233
2487,2023,14,1213
2488,2023,15,1421


In [31]:
sub['deltaSeed'] = sub.apply(delta_seed,axis=1)
sub

,ID,Pred,Season,Team1,Team2,deltaWinPct,deltaSeed
0,2023_1345_1345,0.5,2023,1345,1345,0.000000,0
1,2023_1345_1266,0.5,2023,1345,1266,0.029412,-1
2,2023_1345_1243,0.5,2023,1345,1243,0.134191,-2
3,2023_1345_1397,0.5,2023,1345,1397,0.155971,-3
4,2023_1345_1181,0.5,2023,1345,1181,0.088235,-4
...,...,...,...,...,...,...,...
4619,2023_1224_1433,0.5,2023,1224,1433,-0.181214,4
4620,2023_1224_1233,0.5,2023,1224,1233,-0.181214,3
4621,2023_1224_1213,0.5,2023,1224,1213,-0.043347,2
4622,2023_1224_1421,0.5,2023,1224,1421,-0.168347,1


In [32]:
cwd

'C:\\Users\\C176616\\Documents\\GitHub\\MarchMadness'

Ok, so now we have a trained model. Next we need to find sumission data.

The kaggle competition provides a sample submission.csv file that contains a matchup ID, and a default prediction value. 

In [33]:
sub = pd.read_csv(cwd + '/data_stage2/MSampleSubmissionStage2.csv')
sub

,ID,Pred
0,2023_1345_1345,0.5
1,2023_1345_1266,0.5
2,2023_1345_1243,0.5
3,2023_1345_1397,0.5
4,2023_1345_1181,0.5
...,...,...
4619,2023_1224_1433,0.5
4620,2023_1224_1233,0.5
4621,2023_1224_1213,0.5
4622,2023_1224_1421,0.5


Split this string into Team IDs and year

In [34]:
sub['Season'], sub['Team1'], sub['Team2'] = sub['ID'].str.split('_').str
sub[['Season', 'Team1', 'Team2']] = sub[['Season', 'Team1', 'Team2']].apply(pd.to_numeric)
sub

C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,ID,Pred,Season,Team1,Team2
0,2023_1345_1345,0.5,2023,1345,1345
1,2023_1345_1266,0.5,2023,1345,1266
2,2023_1345_1243,0.5,2023,1345,1243
3,2023_1345_1397,0.5,2023,1345,1397
4,2023_1345_1181,0.5,2023,1345,1181
...,...,...,...,...,...
4619,2023_1224_1433,0.5,2023,1224,1433
4620,2023_1224_1233,0.5,2023,1224,1233
4621,2023_1224_1213,0.5,2023,1224,1213
4622,2023_1224_1421,0.5,2023,1224,1421


Calculate the deltaSeed and deltaWinPct features

In [35]:
sub['deltaWinPct'] = sub.apply(delta_winPct,axis=1)

In [36]:
sub['deltaSeed'] = sub.apply(delta_seed,axis=1)
# sub['deltaMO'] = sub.apply(delta_ord,axis=1)
# sub['deltaWinPct'] = sub.apply(delta_winPct,axis=1)

In [37]:
sub

,ID,Pred,Season,Team1,Team2,deltaWinPct,deltaSeed
0,2023_1345_1345,0.5,2023,1345,1345,0.000000,0
1,2023_1345_1266,0.5,2023,1345,1266,0.029412,-1
2,2023_1345_1243,0.5,2023,1345,1243,0.134191,-2
3,2023_1345_1397,0.5,2023,1345,1397,0.155971,-3
4,2023_1345_1181,0.5,2023,1345,1181,0.088235,-4
...,...,...,...,...,...,...,...
4619,2023_1224_1433,0.5,2023,1224,1433,-0.181214,4
4620,2023_1224_1233,0.5,2023,1224,1233,-0.181214,3
4621,2023_1224_1213,0.5,2023,1224,1213,-0.043347,2
4622,2023_1224_1421,0.5,2023,1224,1421,-0.168347,1


Now, caluclate the rest of our stats. This will take a while.

In [38]:
# cut to slides
rawCols = ['PointsFor','PointsAgainst','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for rawCol in rawCols:
    print("Processing",rawCol)
    sub['delta' + rawCol] = sub.apply(delta_stat,args=(rawCol,),axis=1)

Processing PointsFor
Processing PointsAgainst
Processing FGM
Processing FGA
Processing FGM3
Processing FGA3
Processing FTM
Processing FTA
Processing OR
Processing DR
Processing Ast
Processing TO
Processing Stl
Processing Blk
Processing PF


In [39]:
sub.to_csv("training_set_stage2.csv", index=False)
sub

,ID,Pred,Season,Team1,Team2,deltaWinPct,deltaSeed,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2023_1345_1345,0.5,2023,1345,1345,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2023_1345_1266,0.5,2023,1345,1266,0.029412,-1,-6.852941,-7.852941,-4.176471,...,-4.352941,3.470588,4.176471,3.676471,4.000000,-2.058824,-0.117647,-4.617647,0.470588,-2.558824
2,2023_1345_1243,0.5,2023,1345,1243,0.134191,-2,-2.411765,-6.295956,-0.746324,...,0.150735,-0.676471,-0.726103,2.248162,1.580882,-1.250000,-3.275735,-2.893382,0.836397,-3.981618
3,2023_1345_1397,0.5,2023,1345,1397,0.155971,-3,1.300357,4.704991,-0.104278,...,-2.578431,2.414439,2.526738,-0.485740,0.842246,-1.409091,-0.921569,-3.478610,-0.018717,-3.051693
4,2023_1345_1181,0.5,2023,1345,1181,0.088235,-4,0.558824,-1.205882,-0.558824,...,0.676471,1.647059,2.882353,0.205882,0.323529,0.794118,-1.117647,-0.911765,-0.852941,-1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,2023_1224_1433,0.5,2023,1224,1433,-0.181214,4,2.171727,9.701139,1.278937,...,3.277040,-2.039848,-3.417457,1.967742,-0.168880,1.116698,2.921252,-1.973435,-1.558824,1.224858
4620,2023_1224_1233,0.5,2023,1224,1233,-0.181214,3,-2.769450,7.612903,-2.015180,...,1.188805,0.577799,1.200190,-0.355787,-0.698292,-0.706831,5.480076,-0.502846,-2.558824,1.224858
4621,2023_1224_1213,0.5,2023,1224,1213,-0.043347,2,1.300403,3.925403,1.536290,...,-1.412298,-1.170363,-1.658266,1.873992,-1.736895,2.697581,4.367944,2.373992,-0.125000,-0.920363
4622,2023_1224_1421,0.5,2023,1224,1421,-0.168347,1,0.644153,3.362903,1.192540,...,0.931452,-1.764113,-3.158266,3.123992,-1.986895,1.822581,2.680444,0.842742,-1.031250,1.923387


In [9]:
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2022_1103_1104,0.5,2022,1103,1104,7,0.115927,-10.678427,-12.244960,-4.075605,...,-8.416331,-0.955645,-0.223790,-3.754032,-1.234879,-3.108871,-3.255040,-1.516129,-1.621976,-2.626008
1,2022_1103_1112,0.5,2022,1103,1112,12,-0.202087,-15.268501,-3.368121,-6.860531,...,-0.234345,-1.492410,-0.720114,-2.570209,-4.869070,-8.395636,-2.102467,-1.222011,-2.609108,-0.502846
2,2022_1103_1116,0.5,2022,1103,1116,9,-0.047898,-7.649071,-4.384164,-3.055718,...,0.919844,-2.883675,-2.130987,-1.826002,-2.000978,-2.574780,-1.312805,-2.243402,-1.085044,-0.880743
3,2022_1103_1120,0.5,2022,1103,1120,11,-0.134073,-9.428427,-2.869960,-4.356855,...,-3.697581,-0.299395,0.744960,-2.347782,-2.234879,-3.015121,-0.817540,-3.297379,-4.746976,-2.563508
4,2022_1103_1124,0.5,2022,1103,1124,12,-0.102823,-7.209677,0.536290,-4.638105,...,-1.291331,2.294355,3.463710,-3.097782,0.733871,-4.327621,-1.161290,-3.328629,-0.309476,0.123992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,-9,0.024194,-1.165323,0.860887,0.156250,...,-1.683468,-0.559476,-1.218750,0.996976,-1.804435,0.125000,-2.393145,1.509073,0.743952,1.686492
2274,2022_1458_1463,0.5,2022,1458,1463,-11,0.153504,-1.083426,-3.210234,-0.482759,...,0.163515,0.165740,0.103448,0.385984,-1.293660,-0.827586,-4.235818,-0.709677,-0.365962,-0.618465
2275,2022_1460_1461,0.5,2022,1460,1461,4,-0.143939,3.276515,5.918561,2.580492,...,-3.721591,-0.656250,-1.946023,2.063447,-3.214015,2.912879,1.339015,1.976326,0.846591,-0.766098
2276,2022_1460_1463,0.5,2022,1460,1463,2,-0.014629,3.358412,1.847440,1.941484,...,-1.874608,0.068966,-0.623824,1.452456,-2.703239,1.960293,-0.503657,-0.242424,-0.263323,-3.071055
